In [ ]:
def zoom_class_detection_loose(image_path, class_name, model_path=r"D:\GIT\JioHotstar-Ad-Vision\models\Ad_track.pt", out_path="zoomed_loose.jpg", margin_ratio=0.25):
    """
    Loosely zoom (not tightly crop) on the first detected instance of class_name.

    Args:
        image_path (str): Path to input image.
        class_name (str): Target class name to zoom.
        model_path (str): Path to YOLO model file.
        out_path (str): Path to save cropped image.
        margin_ratio (float): Fraction of box size added as margin (default 0.25 = 25%)
    """
    # Load model
    from ultralytics import YOLO
    import cv2

    model = YOLO(model_path)
    img = cv2.imread(image_path)
    h, w = img.shape[:2]

    # Run detection
    results = model(img)
    boxes = results[0].boxes
    names = results[0].names

    found = False
    for box, cls_id in zip(boxes, boxes.cls.tolist()):
        detected_class = names[int(cls_id)]
        if detected_class == class_name:
            x_min, y_min, x_max, y_max = [int(v) for v in box.xyxy[0].tolist()]
            box_w = x_max - x_min
            box_h = y_max - y_min

            # Loose margin (e.g., 25% extra space)
            delta_w = int(box_w * margin_ratio)
            delta_h = int(box_h * margin_ratio)

            x_min = max(0, x_min - delta_w)
            y_min = max(0, y_min - delta_h)
            x_max = min(w, x_max + delta_w)
            y_max = min(h, y_max + delta_h)
            crop = img[y_min:y_max, x_min:x_max]

            cv2.imwrite(out_path, crop)
            print(f"Loosely cropped {class_name} at {image_path} → {out_path}")
            found = True
            break
    if not found:
        print(f"Class '{class_name}' not found in {image_path}")

# Example usage:
zoom_class_detection_loose(r"D:\GIT\JioHotstar-Ad-Vision\data\1\extracted_frames\Qantas\frame_000736.jpg", "Qantas", model_path="yolov8.pt", out_path="zoomed_loose_qantas.jpg")


FileNotFoundError: [Errno 2] No such file or directory: 'yolov8.pt'